# Portfolio Analysis Example

This notebook demonstrates portfolio-level analysis including:

1. Portfolio Construction
2. Performance Tracking
3. Correlation Analysis
4. Risk Metrics
5. Diversification Analysis
6. Rebalancing Recommendations

---

In [ ]:
# Setup
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

from src.data.fetcher import get_stock_data, get_multiple_stocks
from src.visualization.dashboard import PortfolioDashboard

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("✅ Libraries loaded successfully!")
print("\n📊 Starting Portfolio Analysis...")

## 1. Define Portfolio

Define your portfolio holdings with number of shares for each stock.

In [ ]:
# Portfolio definition
portfolio = {
    'AAPL': 15,   # Apple
    'MSFT': 10,   # Microsoft
    'GOOGL': 5,   # Alphabet
    'AMZN': 8,    # Amazon
    'NVDA': 12,   # NVIDIA
    'JPM': 20,    # JPMorgan
    'JNJ': 15,    # Johnson & Johnson
    'V': 10       # Visa
}

print("Portfolio Holdings:")
print("="*40)
for ticker, shares in portfolio.items():
    print(f"{ticker:6s}: {shares:3d} shares")
print("="*40)

## 2. Fetch Portfolio Data

In [ ]:
# Fetch data for all stocks
tickers = list(portfolio.keys())
start_date = '2022-01-01'

print(f"Fetching data for {len(tickers)} stocks...\n")
stock_data = get_multiple_stocks(tickers, start=start_date)

print(f"✅ Data fetched for all stocks")
print(f"Date range: {stock_data[tickers[0]].index[0].date()} to {stock_data[tickers[0]].index[-1].date()}")

## 3. Portfolio Performance

In [ ]:
# Calculate portfolio values
portfolio_values = {}
for ticker, shares in portfolio.items():
    portfolio_values[ticker] = stock_data[ticker]['Close'] * shares

portfolio_df = pd.DataFrame(portfolio_values)
portfolio_df['Total'] = portfolio_df.sum(axis=1)

# Current values
current_values = portfolio_df.iloc[-1]
initial_values = portfolio_df.iloc[0]

print("\n" + "="*70)
print("💼 PORTFOLIO SUMMARY")
print("="*70)
print(f"\nInitial Portfolio Value: ${initial_values['Total']:,.2f}")
print(f"Current Portfolio Value: ${current_values['Total']:,.2f}")

total_return = ((current_values['Total'] - initial_values['Total']) / initial_values['Total']) * 100
print(f"\nTotal Return: {total_return:+.2f}%")

# Individual holdings
print(f"\n{'Stock':<8} {'Shares':>8} {'Value':>12} {'% Port':>8} {'Return':>10}")
print("-"*60)
for ticker in tickers:
    value = current_values[ticker]
    pct_port = (value / current_values['Total']) * 100
    stock_return = ((current_values[ticker] - initial_values[ticker]) / initial_values[ticker]) * 100
    print(f"{ticker:<8} {portfolio[ticker]:>8d} ${value:>10,.2f} {pct_port:>7.1f}% {stock_return:>9.1f}%")
print("-"*60)
print(f"{'TOTAL':<8} {'':<8s} ${current_values['Total']:>10,.2f} {'100.0%':>7s} {total_return:>9.1f}%")
print("="*70)

In [ ]:
# Plot portfolio value over time
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=portfolio_df.index,
    y=portfolio_df['Total'],
    name='Total Portfolio Value',
    line=dict(color='darkblue', width=3),
    fill='tozeroy'
))

fig.update_layout(
    title='Portfolio Value Over Time',
    xaxis_title='Date',
    yaxis_title='Value ($)',
    template='plotly_white',
    height=500
)

fig.show()

In [ ]:
# Portfolio allocation pie chart
fig = go.Figure(data=[go.Pie(
    labels=tickers,
    values=[current_values[t] for t in tickers],
    hole=0.4
)])

fig.update_layout(
    title='Current Portfolio Allocation',
    height=500
)

fig.show()

## 4. Correlation Analysis

Understand how stocks move together.

In [ ]:
# Calculate daily returns
returns = {}
for ticker in tickers:
    returns[ticker] = stock_data[ticker]['Close'].pct_change()

returns_df = pd.DataFrame(returns)

# Correlation matrix
correlation = returns_df.corr()

print("\n📊 Correlation Matrix:")
print("="*70)
print(correlation)
print("="*70)

In [ ]:
# Visualize correlation heatmap
import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(
    z=correlation.values,
    x=list(correlation.columns),
    y=list(correlation.index),
    colorscale='RdBu',
    zmid=0
)

fig.update_layout(
    title='Portfolio Correlation Heatmap',
    height=600,
    width=700
)

fig.show()

print("\n💡 Interpretation:")
print("  1.0 = Perfect positive correlation")
print("  0.0 = No correlation")
print(" -1.0 = Perfect negative correlation")
print("\n✅ Good diversification: Low correlations between stocks")
print("⚠️  Poor diversification: High correlations (>0.8)")

## 5. Risk Metrics

In [ ]:
# Calculate risk metrics for each stock
risk_metrics = {}

for ticker in tickers:
    ticker_returns = returns_df[ticker].dropna()
    
    # Annualized metrics
    annual_return = ticker_returns.mean() * 252 * 100
    annual_volatility = ticker_returns.std() * np.sqrt(252) * 100
    sharpe_ratio = annual_return / annual_volatility if annual_volatility > 0 else 0
    
    # Max drawdown
    cumulative = (1 + ticker_returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min() * 100
    
    risk_metrics[ticker] = {
        'Annual Return (%)': annual_return,
        'Volatility (%)': annual_volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown (%)': max_drawdown
    }

risk_df = pd.DataFrame(risk_metrics).T

print("\n" + "="*70)
print("📉 RISK METRICS BY STOCK")
print("="*70)
print(risk_df)
print("="*70)

In [ ]:
# Portfolio-level risk
portfolio_returns = portfolio_df['Total'].pct_change().dropna()

portfolio_annual_return = portfolio_returns.mean() * 252 * 100
portfolio_volatility = portfolio_returns.std() * np.sqrt(252) * 100
portfolio_sharpe = portfolio_annual_return / portfolio_volatility if portfolio_volatility > 0 else 0

# Max drawdown
cumulative_port = (1 + portfolio_returns).cumprod()
running_max_port = cumulative_port.expanding().max()
drawdown_port = (cumulative_port - running_max_port) / running_max_port
max_dd_port = drawdown_port.min() * 100

print("\n" + "="*70)
print("📊 PORTFOLIO-LEVEL RISK METRICS")
print("="*70)
print(f"Annual Return: {portfolio_annual_return:.2f}%")
print(f"Volatility: {portfolio_volatility:.2f}%")
print(f"Sharpe Ratio: {portfolio_sharpe:.2f}")
print(f"Max Drawdown: {max_dd_port:.2f}%")
print("="*70)

In [ ]:
# Risk-Return scatter plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=risk_df['Volatility (%)'],
    y=risk_df['Annual Return (%)'],
    mode='markers+text',
    marker=dict(size=15, color=risk_df['Sharpe Ratio'], colorscale='Viridis', showscale=True),
    text=risk_df.index,
    textposition='top center'
))

fig.update_layout(
    title='Risk-Return Profile',
    xaxis_title='Volatility (%)',
    yaxis_title='Annual Return (%)',
    template='plotly_white',
    height=500
)

fig.show()

print("\n💡 Ideal stocks are in the top-left (high return, low risk)")

## 6. Diversification Score

In [ ]:
# Calculate diversification score
def calculate_diversification_score(portfolio_weights, correlation_matrix):
    """
    Score from 0-100 where 100 = perfectly diversified
    """
    # Average pairwise correlation
    n = len(correlation_matrix)
    avg_corr = (correlation_matrix.sum().sum() - n) / (n * (n - 1))
    
    # Concentration (Herfindahl index)
    herfindahl = sum(w**2 for w in portfolio_weights.values())
    
    # Score components
    corr_score = (1 - avg_corr) * 50  # Lower correlation = better
    concentration_score = (1 - herfindahl) / (1 - 1/n) * 50  # Lower concentration = better
    
    total_score = corr_score + concentration_score
    return total_score, avg_corr, herfindahl

# Calculate weights
weights = {ticker: current_values[ticker] / current_values['Total'] for ticker in tickers}

div_score, avg_corr, herf = calculate_diversification_score(weights, correlation)

print("\n" + "="*70)
print("🎯 DIVERSIFICATION ANALYSIS")
print("="*70)
print(f"\nDiversification Score: {div_score:.1f}/100")
print(f"\nComponents:")
print(f"  Average Correlation: {avg_corr:.3f}")
print(f"  Concentration Index: {herf:.3f}")

if div_score >= 80:
    rating = "Excellent ✅"
elif div_score >= 60:
    rating = "Good ✅"
elif div_score >= 40:
    rating = "Fair ⚠️"
else:
    rating = "Poor ❌"

print(f"\nRating: {rating}")
print("="*70)

## 7. Rebalancing Recommendations

In [ ]:
# Target equal weighting
target_weight = 1 / len(tickers)

print("\n" + "="*70)
print("⚖️ REBALANCING RECOMMENDATIONS (Equal Weight Target)")
print("="*70)
print(f"\nTarget Weight per Stock: {target_weight*100:.1f}%\n")
print(f"{'Stock':<8} {'Current %':>12} {'Target %':>12} {'Difference':>12} {'Action':>15}")
print("-"*70)

for ticker in tickers:
    current_pct = weights[ticker] * 100
    target_pct = target_weight * 100
    diff = current_pct - target_pct
    
    if abs(diff) < 2:
        action = "✅ Balanced"
    elif diff > 0:
        action = f"🔴 Reduce {abs(diff):.1f}%"
    else:
        action = f"🟢 Increase {abs(diff):.1f}%"
    
    print(f"{ticker:<8} {current_pct:>11.1f}% {target_pct:>11.1f}% {diff:>11.1f}% {action:>15s}")

print("="*70)
print("\n💡 Rebalancing helps maintain target allocation and manage risk.")

## 8. Interactive Dashboard

In [ ]:
# Use the portfolio dashboard
dashboard = PortfolioDashboard(portfolio, start_date=start_date)
dashboard.show()

## Summary

This portfolio analysis covered:

✅ **Portfolio Construction** - Defined holdings and calculated values  
✅ **Performance Tracking** - Total and individual returns  
✅ **Correlation Analysis** - Measured diversification  
✅ **Risk Metrics** - Volatility, Sharpe ratio, max drawdown  
✅ **Diversification Score** - Quantified portfolio diversity  
✅ **Rebalancing Recommendations** - Suggested adjustments  

---

### Key Insights:

1. **Diversification is crucial** - Low correlations reduce portfolio risk
2. **Regular rebalancing** maintains target allocation
3. **Monitor risk metrics** - Not just returns
4. **Consider different sectors** - Tech, finance, healthcare, etc.
5. **Review periodically** - Markets change, allocations drift

### Next Steps:

- Update your portfolio holdings in cell 2
- Run the analysis monthly or quarterly
- Consider sector and geographic diversification
- Implement dollar-cost averaging
- Consult with financial advisors

**Use this as a template for tracking your real portfolio!**